# Getting started with Spectral Radius Optimization

This notebook contains examples of how to solve Spectral Radius Optimization problem.

For more details, please check the documentation website https://pygranso.readthedocs.io/en/latest/

1. Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
import os,sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/PyGRANSO')
from pygranso import pygranso
from pygransoStruct import Options, Data, GeneralStruct 
import scipy.io
from torch import linalg as LA

2. Specify torch device, and read the data from provided file

In [2]:
device = torch.device('cpu')

# currentdir = os.path.dirname(os.path.realpath(__file__))
file = "/home/buyun/Documents/GitHub/PyGRANSO/examples/data/spec_radius_opt_data.mat"
mat = scipy.io.loadmat(file)
mat_struct = mat['sys']
mat_struct = mat_struct[0,0]
A = torch.from_numpy(mat_struct['A']).to(device=device, dtype=torch.double)
B = torch.from_numpy(mat_struct['B']).to(device=device, dtype=torch.double)
C = torch.from_numpy(mat_struct['C']).to(device=device, dtype=torch.double)
p = B.shape[1]
m = C.shape[0]
stability_margin = 1

3. Spceify optimization variables and corresponding objective and constrained function.

Note: please strictly follow the format of evalObjFunction and combinedFunction, which will be used in the PyGRANSO main algortihm. 

In [3]:
# variables and corresponding dimensions.
var_in = {"X": [p,m] }


def evalObjFunction(X_struct):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X
    X.requires_grad_(True)

    # objective function
    M           = A + B@X@C
    [D,_]       = LA.eig(M)
    f = torch.max(D.imag)
    return f

def combinedFunction(X_struct):
    # user defined variable, matirx form. torch tensor
    X = X_struct.X
    X.requires_grad_(True)

    # objective function
    M           = A + B@X@C
    [D,_]       = LA.eig(M)
    f = torch.max(D.imag)

    # inequality constraint, matrix form
    ci = GeneralStruct()
    ci.c1 = torch.max(D.real) + stability_margin

    # equality constraint 
    ce = None
    
    return [f,ci,ce]

obj_eval_fn = lambda X_struct : evalObjFunction(X_struct)
comb_fn = lambda X_struct : combinedFunction(X_struct)

4. Specify user-defined options for PyGRANSO algorithm

In [4]:
opts = Options()
opts.QPsolver = 'osqp' 
opts.maxit = 200
opts.x0 = torch.zeros(p*m,1).to(device=device, dtype=torch.double)
opts.print_level = 1
opts.print_frequency = 10
opts.limited_mem_size = 40

4. Run main algorithm

In [5]:
start = time.time()
soln = pygranso(combinedFunction = comb_fn, objEvalFunction = obj_eval_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ══════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible   ║
║  interface, as defined by osqp and Gurobi...                                        ║
╚═════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
Problem specifications:                                                                                          ║ 
 # of variables                     :   200                                                                      ║ 
 # of inequality constraints        :     1                                                                      ║ 
 # of equality constraints          :     0                                                                      ║ 
═══════════════════════════════════════════════════════════════════

/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/osqp/utils.py:122: UserWarning: Converting sparse A to a CSC (compressed sparse column) matrix. (It may take a while...)
  warn("Converting sparse A to a CSC " +


  10 ║ 1.000000 │  14.3591621233 ║  12.9268286638 ║ 1.432333 │   -  ║ S  │     1 │ 1.000000 ║     1 │ 0.035476   ║ 
  20 ║ 1.000000 │  13.7108075289 ║  12.6560751464 ║ 1.054732 │   -  ║ S  │     2 │ 0.500000 ║     1 │ 0.039916   ║ 
  30 ║ 1.000000 │  12.9780969354 ║  12.2693532882 ║ 0.708744 │   -  ║ S  │     5 │ 0.062500 ║     1 │ 0.044711   ║ 
  40 ║ 1.000000 │  12.6278337430 ║  12.0378971738 ║ 0.589937 │   -  ║ S  │     4 │ 0.125000 ║     1 │ 0.022880   ║ 


/home/buyun/Documents/GitHub/PyGRANSO/private/bfgsHessianInverseLimitedMem.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.rho = torch.hstack((self.rho[:,1:], torch.tensor(rho_new)))


  50 ║ 0.810000 │  10.0998832396 ║  11.9737990401 ║ 0.401106 │   -  ║ S  │     4 │ 0.125000 ║     1 │ 0.022497   ║ 
  60 ║ 0.810000 │  9.85470307548 ║  11.8691995412 ║ 0.240651 │   -  ║ S  │     4 │ 0.125000 ║     1 │ 0.019422   ║ 
  70 ║ 0.810000 │  9.76355751695 ║  11.7944872776 ║ 0.210023 │   -  ║ S  │     8 │ 0.007812 ║     1 │ 0.175698   ║ 
  80 ║ 0.810000 │  9.66853158992 ║  11.7506269498 ║ 0.150524 │   -  ║ S  │     6 │ 0.031250 ║     1 │ 0.018261   ║ 
  90 ║ 0.810000 │  9.60157661057 ║  11.7237537653 ║ 0.105336 │   -  ║ S  │     7 │ 0.015625 ║     1 │ 0.141248   ║ 
 100 ║ 0.531441 │  6.27510636475 ║  11.6973832178 ║ 0.058637 │   -  ║ S  │    10 │ 0.001953 ║     1 │ 0.519602   ║ 
 110 ║ 0.531441 │  6.26154786174 ║  11.6752864747 ║ 0.056822 │   -  ║ S  │    12 │ 0.001465 ║     1 │ 0.197857   ║ 
 120 ║ 0.348678 │  4.10994349086 ║  11.6996971452 ║ 0.030511 │   -  ║ S  │    15 │ 4.27e-04 ║     1 │ 64.43392   ║ 
 130 ║ 0.348678 │  4.07047561607 ║  11.6740100561 ║ 0.000000 │   -  ║ S 